<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/MultivariateWienerPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Upgrade to TensorFlow nightly
!pip install --upgrade tf-nightly

     |████████████████████████████████| 398.7MB 45kB/s 
     |████████████████████████████████| 1.3MB 41.9MB/s 
     |████████████████████████████████| 11.9MB 25.1MB/s 
     |████████████████████████████████| 3.8MB 33.8MB/s 
     |████████████████████████████████| 14.5MB 149kB/s 
     |████████████████████████████████| 471kB 41.4MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: grpcio 1.33.2
    Uninstalling grpcio-1.33.2:
      Successfully uninstalled grpcio-1.33.2
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.1

In [ ]:
import tensorflow as tf

In [ ]:
from scipy.special import erf
import numpy as np
import time

In [ ]:
x = tf.math.sobol_sample(10,10)

In [ ]:
import numpy as np

In [ ]:
class BrownianBridge():
    def __init__(self, number_time_steps):
        super(BrownianBridge, self).__init__()

        self.number_time_steps = number_time_steps

        self.left_index = np.zeros(number_time_steps, dtype=int)
        self.right_index = np.zeros(number_time_steps, dtype=int)
        self.bridge_index = np.zeros(number_time_steps, dtype=int)
        self.left_weight = np.zeros(number_time_steps)
        self.right_weight = np.zeros(number_time_steps)
        self.std_dev = np.zeros(number_time_steps)

        self._map = np.zeros(number_time_steps, dtype=int)

        self._map[-1] = 1
        self.bridge_index[0] = number_time_steps - 1
        self.std_dev[0] = tf.math.sqrt(1.0 * number_time_steps)
        self.left_weight[0] = 0
        self.right_weight[0] = 0

        j=0
        for i in range(1,number_time_steps):
            while self._map[j] == True:
                j = j + 1
            k = j
            while self._map[k] == False:
                k = k + 1
            l = j+((k-1-j)>>1)
            self._map[l]=i
            self.bridge_index[i]=l
            self.left_index[i]=j
            self.right_index[i]=k
            self.left_weight[i]=(k-l)/(k+1-j)
            self.right_weight[i]=(1+l-j)/(k+1-j)
            self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
            j=k+1
            if j>=number_time_steps:
                j=0

    @tf.function(jit_compile=True)
    def buildPath(self, z, number):
        path = tf.unstack(tf.zeros(shape=(self.number_time_steps,number)))
        path[-1] = self.std_dev[0]*z[0];
        for i in np.arange(1,self.number_time_steps):
            j = self.left_index[i]
            k = self.right_index[i]
            l = self.bridge_index[i]
            #if j > 0:
            #    path[l] = self.left_weight[i] * path[j-1] + self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
            #else:
            path[l] = self.right_weight[i] * path[k] + self.std_dev[i] * z[i]

        return path

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import math

In [ ]:
number_time_steps = 128
number = 3

left_index = np.zeros(number_time_steps, dtype=int)
right_index = np.zeros(number_time_steps, dtype=int)
bridge_index = np.zeros(number_time_steps, dtype=int)
left_weight = np.zeros(number_time_steps)
right_weight = np.zeros(number_time_steps)
std_dev = np.zeros(number_time_steps)

_map = np.zeros(number_time_steps, dtype=int)

_map[-1] = 1
bridge_index[0] = number_time_steps - 1
std_dev[0] = math.sqrt(1.0 * number_time_steps)
left_weight[0] = 0
right_weight[0] = 0

j=0
for i in range(1,number_time_steps):
  while _map[j] == True:
    j = j + 1
  k = j
  while _map[k] == False:
    k = k + 1
  l = j+((k-1-j)>>1)
  _map[l]=i
  bridge_index[i]=l
  left_index[i]=j
  right_index[i]=k
  left_weight[i]=(k-l)/(k+1-j)
  right_weight[i]=(1+l-j)/(k+1-j)
  std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
  j=k+1
  if j>=number_time_steps:
    j=0

In [ ]:
bridge_index_t = tf.constant(bridge_index)
right_index_t = tf.constant(right_index)
left_index_t = tf.constant(left_index)
left_weight_t = tf.constant(left_weight)
right_weight_t = tf.constant(right_weight)
std_dev_t = tf.constant(std_dev)

print(left_index_t)

NameError: ignored

In [ ]:
#@tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None), dtype=tf.float64), tf.TensorSpec(shape=(), dtype=tf.int32)))
@tf.function(jit_compile=True)
def test(z, number):
  print("Tracing with", number)

  path = tf.unstack(tf.zeros(shape=(8,3),dtype=tf.dtypes.float64))
  path[-1] = z[0]
  print(path)
  print(z)
  for i in range(number-1):
    print(i)  
    print(z[i])
    #print(path[i])
    #tmp = z[i]
    path[i] = z[i]

  return path

In [ ]:
number = tf.constant(3)

x = tf.zeros(shape=(3,1),dtype=tf.dtypes.float64)

test(x,number)

print(x)

Tracing with Tensor("number:0", shape=(), dtype=int32)
[<tf.Tensor 'unstack:0' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:1' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:2' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:3' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:4' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:5' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:6' shape=(3,) dtype=float64>, <tf.Tensor 'strided_slice:0' shape=(1,) dtype=float64>]
Tensor("z:0", shape=(3, 1), dtype=float64)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/strided_slice:0", shape=(1,), dtype=float64)


TypeError: ignored

In [ ]:
print(tf.autograph.to_code(test.python_function))

In [ ]:
print(test.pretty_printed_concrete_signatures())

test(z, number=3)
  Args:
    z: float64 Tensor, shape=(3, 1)
  Returns:
    [<1>, <2>, <3>, <4>, <5>, <6>, <7>, <8>]
      <1>: float64 Tensor, shape=(1,)
      <2>: float64 Tensor, shape=(1,)
      <3>: float64 Tensor, shape=(3,)
      <4>: float64 Tensor, shape=(3,)
      <5>: float64 Tensor, shape=(3,)
      <6>: float64 Tensor, shape=(3,)
      <7>: float64 Tensor, shape=(3,)
      <8>: float64 Tensor, shape=(1,)


In [ ]:
#@tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None), dtype=tf.float64), 
#                                                tf.TensorSpec(shape=(), dtype=tf.int32),
#                                                tf.TensorSpec(shape=(), dtype=tf.int32),
#                                                tf.TensorSpec(shape=(None), dtype=tf.int64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.int64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.int64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.float64)))
@tf.function(jit_compile=True)
def buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):
  print("Tracing with", number)
  path = tf.unstack(tf.zeros(shape=(number_time_steps, number),dtype=tf.dtypes.float64))
  path[-1] = std_dev[0]*z[0];
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
#    x = z[i]
#    print("x")
#    print(x)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    if j > 0:
      path[l] = left_weight[i] * path[j-1] + right_weight[i] * path[k] + self.std_dev[i] * z[i]
    else:
      path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

TypeError: ignored

In [ ]:
samples = 8
dim = 128

x = tf.math.sobol_sample(dim,samples,dtype=tf.dtypes.float64)
z = tf.transpose(tf.math.erf(x))

In [ ]:
path = buildPath(z, samples, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

TypeError: ignored

In [ ]:
print(buildPath.pretty_printed_concrete_signatures())

In [ ]:
print(tf.autograph.to_code(buildPath.python_function))

AttributeError: ignored

In [ ]:
bridge = BrownianBridge(2)

In [ ]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  200000#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float32)
    z = tf.transpose(tf.math.erf(x))
    #w = buildPath(z, number, dim, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    #w = buildPath(z, number, dim, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
    #print(w[2])
time_end = time.time()

print(w)

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.40128326416015625
[<tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0.

In [ ]:
import torch

In [ ]:
@torch.jit.script
def foo(x, number: int):
  print(number)
  z = 0.
  for i in range(0,number):
    tmp = x[i]
    z = z + tmp

# See the compiled graph as Python code
print(foo.code)

def foo(x: Tensor,
    number: int) -> None:
  print(number)
  z = 0.
  for _0 in range(torch.__range_length(0, number, 1)):
    i = torch.__derive_index(_0, 0, 1)
    tmp = torch.select(x, 0, i)
    z0 = torch.add(tmp, z, 1)
    z = annotate(float, z0)
  return None



In [ ]:
# Call the function using the TorchScript interpreter
x = torch.zeros(11)

foo(x, 10)

10


In [ ]:
@torch.jit.script
def buildPath(path, z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
  path[-1] = std_dev[0]*z[0]
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
    lw = left_weight[i]
    rw = right_weight[i]
    sd = std_dev[i]
    #print(lw)
    if j > 0:
      path[l] = path[j-1] * lw + path[k] * rw + z[i] * sd
    else:
      path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [ ]:
#scripted_func = torch.jit.script(buildPath)

In [ ]:
print(buildPath.code)

def buildPath(path: Tensor,
    z: Tensor,
    number: int,
    number_time_steps: int,
    left_index: Tensor,
    right_index: Tensor,
    bridge_index: Tensor,
    left_weight: Tensor,
    right_weight: Tensor,
    std_dev: Tensor) -> Tensor:
  _0 = torch.mul(torch.select(std_dev, 0, 0), torch.select(z, 0, 0))
  _1 = torch.copy_(torch.select(path, 0, -1), _0, False)
  _2 = torch.__range_length(1, number_time_steps, 1)
  for _3 in range(_2):
    i = torch.__derive_index(_3, 1, 1)
    j = torch.select(left_index, 0, i)
    _4 = annotate(int, j)
    k = torch.select(right_index, 0, i)
    _5 = annotate(int, k)
    l = torch.select(bridge_index, 0, i)
    _6 = annotate(int, l)
    lw = torch.select(left_weight, 0, i)
    rw = torch.select(right_weight, 0, i)
    sd = torch.select(std_dev, 0, i)
    if torch.gt(_4, 0):
      _7 = torch.select(path, 0, torch.sub(_4, 1))
      _8 = torch.mul(_7, lw)
      _9 = torch.mul(torch.select(path, 0, _5), rw)
      _10 = torch.add(_8, _9, alpha=1)


In [ ]:
path = torch.zeros(size=(128,2))

x = torch.randn(128,2)

left_index_t = torch.tensor(left_index)
right_index_t = torch.tensor(right_index)
bridge_index_t = torch.tensor(bridge_index)
left_weight_t = torch.tensor(left_weight)
right_weight_t = torch.tensor(right_weight)
std_dev_t = torch.tensor(std_dev)

path = buildPath(path, x, 2, 128, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

In [ ]:
scripted_func.graph_for(path, x, 2, 8, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

NameError: ignored

In [ ]:
sobol_engine =  torch.quasirandom.SobolEngine(2)
sobol_engine.draw(3)

tensor([[0.5000, 0.5000],
        [0.7500, 0.2500],
        [0.2500, 0.7500]])

In [ ]:
import time

In [ ]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

path = torch.zeros(size=(dim,number))

sobol_engine =  torch.quasirandom.SobolEngine(dim)

# First run (includes graph optimization time)
time_start = time.time()
x = sobol_engine.draw(number)
y = torch.transpose(torch.erf(x),0,1)
path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
x = sobol_engine.draw(number)
y = torch.transpose(torch.erf(x),0,1)
path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

# First run (includes graph optimization time)
time_start = time.time()
x = torch.randn(size=(dim,number))
#y = torch.transpose(torch.erf(x),0,1)
#path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Third time on a CPU: ", time_price_cpu)


First time on a CPU:  0.5058045387268066
Second time on a CPU:  0.5059146881103516
Third time on a CPU:  0.2710602283477783


In [ ]:
import torch
import torchvision.models as models
import torch.autograd.profiler as profiler

In [ ]:
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

path = torch.zeros(size=(dim,number), dtype=float)

with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        x = sobol_engine.draw(number)
        y = torch.transpose(torch.erf(x-0.5),0,1)
        path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)


In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
              model_inference         0.27%       1.913ms       100.00%     712.723ms     712.723ms             1  
                    buildPath         7.40%      52.736ms        82.34%     586.909ms     586.909ms             1  
                    aten::mul        52.40%     373.460ms        52.78%     376.227ms       1.003ms           375  
                    aten::add        11.65%      83.040ms        16.02%     114.209ms     462.383us           247  
                    aten::erf         6.04%      43.024ms        12.07%      86.032ms      43.016ms             2  
                  aten::copy_         8.70%      62.007ms         8.70% 

In [ ]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))


-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                   Input Shapes  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------  
              model_inference         6.57%       1.723ms        99.88%      26.192ms      26.192ms             1                                             []  
                    buildPath         3.51%     920.948us        60.50%      15.867ms      15.867ms             1  [[8, 200000], [8, 200000], [], [], [8], [8],   
                    aten::add        14.76%       3.872ms        21.80%       5.718ms     519.818us            11               [[], [], [200000], [200000], []]  
     aten::_sobol_engi

In [ ]:
prof.export_chrome_trace("trace.json")

In [4]:
import torch

In [5]:
sobol_engine =  torch.quasirandom.SobolEngine(4)
x = sobol_engine.draw(3)

In [6]:
x = torch.transpose(x,0,1)
print(x.size())
x[:,0]

torch.Size([4, 3])


tensor([0.5000, 0.5000, 0.5000, 0.5000])

In [12]:
x[0,:]

tensor([0.5000, 0.7500, 0.2500])

In [17]:
y = torch.reshape(x,shape=(2,2,3))
y[1]

tensor([[0.5000, 0.7500, 0.2500],
        [0.5000, 0.2500, 0.7500]])

In [18]:
print(y.size())

torch.Size([2, 2, 3])


In [ ]:
y[:][:][0]

tensor([[0.5000, 0.7500]])

In [ ]:
m = torch.zeros(size=(2,2))
m[0,0] = 1
m[1,0] = 0
m[1,1] = 1

In [ ]:
w = torch.matmul(m,y[0,:,:])

In [ ]:
w

tensor([[0.5000, 0.7500, 0.2500],
        [0.5000, 0.2500, 0.7500]])

# Univariate Wiener Path Construction

In [98]:
import time
import math
import numpy as np

In [99]:
class UnivariateBrownianBridge():
  def __init__(self, number_time_steps):
    self.number_time_steps = number_time_steps
    number = 3

    self.left_index = torch.zeros(number_time_steps, dtype=int)
    self.right_index = torch.zeros(number_time_steps, dtype=int)
    self.bridge_index = torch.zeros(number_time_steps, dtype=int)
    self.left_weight = torch.zeros(number_time_steps)
    self.right_weight = torch.zeros(number_time_steps)
    self.std_dev = torch.zeros(number_time_steps)

    self._map = torch.zeros(number_time_steps, dtype=int)

    self._map[-1] = 1
    self.bridge_index[0] = number_time_steps - 1
    self.std_dev[0] = torch.sqrt(torch.tensor(1.0) * number_time_steps)
    self.left_weight[0] = 0
    self.right_weight[0] = 0

    j=0
    for i in range(1,number_time_steps):
      while self._map[j] == True:
        j = j + 1
      k = j
      while self._map[k] == False:
        k = k + 1
      l = j+((k-1-j)>>1)
      self._map[l]=i
      self.bridge_index[i]=l
      self.left_index[i]=j
      self.right_index[i]=k
      self.left_weight[i]=(k-l)/(k+1-j)
      self.right_weight[i]=(1+l-j)/(k+1-j)
      self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
      j=k+1
      if j>=number_time_steps:
        j=0
        
  @torch.jit.script
  def buildPath(path, z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path[-1] = std_dev[0]*z[0]
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      lw = left_weight[i]
      rw = right_weight[i]
      sd = std_dev[i]
      if j > 0:
        path[l] = path[j-1] * lw + path[k] * rw + z[i] * sd
      else:
        path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

In [100]:
brownian = UnivariateBrownianBridge(8)

In [101]:
path = torch.zeros(size=(8,2))

x = torch.randn(8,2)

UnivariateBrownianBridge.buildPath(path, x, 2, 8, brownian.left_index, brownian.right_index, brownian.bridge_index, brownian.left_weight, brownian.right_weight, brownian.std_dev)

In [108]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
numberTimeSteps =  128#@param {type:"integer"}
numberSimulation =  200000#@param {type:"integer"}


# First run (includes graph optimization time)
time_start = time.time()
path = torch.zeros(size=(numberTimeSteps,numberSimulation))

brownian = UnivariateBrownianBridge(numberTimeSteps)

sobol_engine =  torch.quasirandom.SobolEngine(numberTimeSteps)

x = sobol_engine.draw(number)
y = torch.transpose(torch.erf(x),0,1)
BrownianBridge.buildPath(path, y, numberSimulation, numberTimeSteps, brownian.left_index, brownian.right_index, brownian.bridge_index, brownian.left_weight, brownian.right_weight, brownian.std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

First time on a CPU:  0.5597996711730957


# MultivariateBrownian

In [114]:
numberSimulation = 3
numberTimeSteps = 2
numberStates = 2
dim = numberTimeSteps * numberStates

In [115]:
sobol_engine =  torch.quasirandom.SobolEngine(dim)
x = sobol_engine.draw(numberSimulation)
x = torch.transpose(x,0,1)

In [116]:
y = torch.reshape(x,shape=(numberTimeSteps,numberStates,numberSimulation))

In [120]:
m = torch.zeros(size=(numberStates,numberStates))
m[0,0] = 1
m[1,0] = 1
m[1,1] = 0

In [121]:
w = torch.matmul(m,y[0,:,:])

In [122]:
w

tensor([[0.5000, 0.7500, 0.2500],
        [0.5000, 0.7500, 0.2500]])